> **Tip**: Welcome to the Investigate a Dataset project! You will find tips in quoted sections like this to help organize your approach to your investigation. Before submitting your project, it will be a good idea to go back through your report and remove these sections to make the presentation of your work as tidy as possible. First things first, you might want to double-click this Markdown cell and change the title so that it reflects your dataset and investigation.

# Project: Analyzing Appointment No-Shows

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

> **Tip**: In this section of the report, provide a brief introduction to the dataset you've selected for analysis. At the end of this section, describe the questions that you plan on exploring over the course of the report. Try to build your report around the analysis of at least one dependent variable and three independent variables.
>
> If you haven't yet selected and downloaded your data, make sure you do that first before coming back here. If you're not sure what questions to ask right now, then make sure you familiarize yourself with the variables and the dataset context for ideas of what to explore.

In [128]:
import pandas as pd
from datetime import datetime

<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you document your steps carefully and justify your cleaning decisions.

### General Properties

In [129]:
df = pd.read_csv('noshowappointments-kagglev2-may-2016.csv')

In [130]:
#df.info()

In [131]:
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).

### Data Cleaning

There are no null values, but there are two changes to make:<br>
1. Convert the No-Show appointment column to a more readable, intuitive name and value. I will change it from the negative to the positive by renaming the column to "ApptResult" with each cooresponding value as "Present" or "Missed."<br>
2. To utilize the dates provided, I will parse out the month and the day for both the scheduled date and the appointment date.<br>
3. Remove same-day / walk-in appointments.<br>

Upon making these changes, I will export to a new, cleaned CSV file for future use.

#### Clean the No-show column

I want to first change the column name from No-show to ApptResult. This column name will be more intuitive to interpret.

In [132]:
df.rename(columns = {'No-show' : 'ApptResult'} , inplace=True)

Once we rename the column from No-show (which is in the negative), we need to rename the ApptResult column values from a "Yes" or "No" value to "Missed" or "Present", respectively.

In [133]:
df['ApptResult'].replace("No", "Present", inplace=True)
df['ApptResult'].replace("Yes", "Missed", inplace=True)

#### Clean the date columns and add a new column that calculates the difference

Convert the AppointmentDate and ScheduledDate to datetime format

In [134]:
#scheduled_date = datetime.strptime(df['ScheduledDay'][1], "%Y-%m-%dT%H:%M:%SZ")
scheduled_date = datetime.strptime(df['ScheduledDay'][10][:10], "%Y-%m-%d")
appointment_date = datetime.strptime(df['AppointmentDay'][10][:10], "%Y-%m-%d")
date_difference = appointment_date - scheduled_date
print("Scheduled Date: {}".format(scheduled_date))
print("Appointment Date: {}".format(appointment_date))
print("Time Between Dates: {}".format(date_difference))

Scheduled Date: 2016-04-27 00:00:00
Appointment Date: 2016-04-29 00:00:00
Time Between Dates: 2 days, 0:00:00


In [135]:
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'], format="%Y-%m-%d").dt.date
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'], format="%Y-%m-%d").dt.date

Add a column to the dataframe "DateDifference" to the dataframe

In [136]:
df['DateDifference'] = df['AppointmentDay'] - df['ScheduledDay']

Retain rows where DateDifference is greater than 0. We can assume that these are walk-in appointments, and appointments scheduled after the appointment date must be a data-entry error.

In [137]:
df = df[df['DateDifference'] > '0 days']

In [144]:
#df.info()
#df.head()
df.groupby('ApptResult')['Age'].describe()

,count,mean,std,min,25%,50%,75%,max
ApptResult,,,,,,,,
Missed,20522.0,34.835299,21.954582,0.0,17.0,33.0,51.0,115.0
Present,51437.0,39.965706,23.140047,0.0,21.0,41.0,58.0,115.0


#### Export the cleaned data to a new CSV

In [142]:
df.to_csv('cleaned-noshowappointments.csv', index=False)

In [90]:
df = pd.read_csv('cleaned-noshowappointments.csv')

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [88]:
df[df['ApptResult'] == 'Yes'].query('SMS_received==0').describe()
df.query('Handcap==1 and SMS_received==1')['ApptResult'].value_counts()

Present    363
Missed     126
Name: ApptResult, dtype: int64

### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!